In [1]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering

In [3]:
len_data_arr = []
results_data_arr_agglo = []
results_data_arr_bunch = []

bunch_bool = False
bunch_hypertune = ['turbomqincrw', 'turbomqincr', 'turbomqw', 'turbomq', 'basicmq']

for root, dirs, files in os.walk('Results_19112020'):
    for file in files:
        #print(file)
        #print(file.split('_')[-1])
        if file.split('_')[-1] == 'len.txt':
            print(file, 'len')
            proj_len_file = open('Results_19112020/' + file)
            proj_name = file.split('_')[0]
            proj_len = proj_len_file.readline().replace('\n','')
            len_data_arr.append([proj_name, proj_len])
            proj_len_file.close()
        elif file.split('_')[-1] == 'results.txt':
            #print(file, 'results')
            proj_result_file = open('Results_19112020/' + file)
            proj_name = file.split('_')[0]
            
            initial_line = 2 - 1
            initial_line_result = 5 - 1
            tmp_arr = []
            
            for i, line in enumerate(proj_result_file):
                
                if i == initial_line:
                    line = line.replace('MoJo', '')
                    line = line.replace('.rsf', '')
                    line = line.replace('(', '')
                    line = line.replace(')', '')
                    line = line.split('_')
                    name = line[1].split('/')[-1]
                    #version = line[0].split('-')[-1]
                    version = line[2]
                    
                    ## If gathers results for bunch
                    if line[3] == 'exhaustive' or line[3] == 'hillclimbing' or line[3] == 'ga':
                        
                        if line[4] in bunch_hypertune:
                            bunch_bool = True
                            method = line[3]
                            calculator = line[4]
                            tmp_arr.append(name)
                            tmp_arr.append(version)
                            tmp_arr.append(method)
                            tmp_arr.append(calculator)

                        initial_line += 5

                    
                    
                    ## Else gathers results for agglomerative
                    else:
                        agglo_bool = True
                        n_cluster = line[3]
                        affinity = line[4]
                        linkage = line[5]
                        #print(line)
                        #print('Version', version)
                        initial_line += 5
                        tmp_arr.append(name)
                        tmp_arr.append(version)
                        tmp_arr.append(n_cluster)
                        tmp_arr.append(affinity)
                        tmp_arr.append(linkage)
                        
                elif i == initial_line_result:
                    n_MoJo = line.split(' ')[-1].split('\n')[0]

                    #print(line)
                    initial_line_result += 5
                    tmp_arr.append(n_MoJo)
                    #tmp_arr.append((1 - (float(n_MoJo) / float(num_class))) * 100)
                    
                    if agglo_bool == True:
                        results_data_arr_agglo.append(tmp_arr)
                        agglo_bool = False
                    elif bunch_bool == True:
                        results_data_arr_bunch.append(tmp_arr)
                        bunch_bool = False
                    tmp_arr = []
                    
                    
                
                

aws-aws-sdk-java-v2_max_len.txt len
bdew-neiaddons_max_len.txt len
bkromhout-realm-java_max_len.txt len
btraceio-btrace_max_len.txt len
bytedeco-javacpp_max_len.txt len
codecentric-spring-boot-admin_max_len.txt len
codenvy-legacy-che-plugins_max_len.txt len
coobird-thumbnailator_max_len.txt len
cryptomator-cryptomator_max_len.txt len
dropwizard-dropwizard_max_len.txt len
dropwizard-metrics_max_len.txt len
evant-gradle-retrolambda_max_len.txt len
facebook-facebook-android-sdk_max_len.txt len
facebook-facebook-java-business-sdk_max_len.txt len
facebook-fresco_max_len.txt len
facebook-litho_max_len.txt len
facebook-react-native-fbsdk_max_len.txt len
flyway_flyway_max_len.txt len
google-cdep_max_len.txt len
google-dagger_max_len.txt len
google-error-prone_max_len.txt len
google-exoplayer_max_len.txt len
google-gitiles_max_len.txt len
google-openrtb-doubleclick_max_len.txt len
google-openrtb_max_len.txt len
grpc-grpc-java_max_len.txt len
havarunner-havarunner_max_len.txt len
hazelcast-hazel

In [4]:
pd.DataFrame(results_data_arr_agglo)

,0,1,2,3,4,5
0,aws-aws-sdk-java-v2,2.9.9,5,euclidean,complete,1280
1,aws-aws-sdk-java-v2,2.9.9,5,euclidean,average,1695
2,aws-aws-sdk-java-v2,2.9.9,5,euclidean,single,644
3,aws-aws-sdk-java-v2,2.9.9,5,l1,complete,1070
4,aws-aws-sdk-java-v2,2.9.9,5,l1,average,695
...,...,...,...,...,...,...
63073,web3j-web3j,v4.5.2,25,manhattan,average,66
63074,web3j-web3j,v4.5.2,25,manhattan,single,59
63075,web3j-web3j,v4.5.2,25,cosine,complete,205
63076,web3j-web3j,v4.5.2,25,cosine,average,251


In [5]:
pd.DataFrame(results_data_arr_bunch)

,0,1,2,3,4
0,cryptomator-cryptomator,1.5.6,hillclimbing,turbomqincrw,102
1,cryptomator-cryptomator,1.5.6,hillclimbing,basicmq,109
2,cryptomator-cryptomator,1.5.6,hillclimbing,turbomqincr,120
3,cryptomator-cryptomator,1.5.6,hillclimbing,turbomq,103
4,cryptomator-cryptomator,1.5.6,hillclimbing,turbomqw,116
...,...,...,...,...,...
9706,web3j-web3j,v4.5.2,ga,turbomqincrw,360
9707,web3j-web3j,v4.5.2,ga,basicmq,336
9708,web3j-web3j,v4.5.2,ga,turbomqincr,321
9709,web3j-web3j,v4.5.2,ga,turbomq,348


In [6]:
pd.DataFrame(len_data_arr)

,0,1
0,aws-aws-sdk-java-v2,2168
1,bdew-neiaddons,38
2,bkromhout-realm-java,352
3,btraceio-btrace,495
4,bytedeco-javacpp,154
...,...,...
72,swagger-api-swagger-core,763
73,testcontainers-testcontainers-java,151
74,uber-NullAway,106
75,voxeolabs-moho,639


In [7]:
agglo_results = pd.DataFrame(results_data_arr_agglo)
agglo_results.columns = ['project_name', 'project_version', 'cluster_division', 'affinity', 'linkage', 'MoJo_results']
agglo_results.head()

,project_name,project_version,cluster_division,affinity,linkage,MoJo_results
0,aws-aws-sdk-java-v2,2.9.9,5,euclidean,complete,1280
1,aws-aws-sdk-java-v2,2.9.9,5,euclidean,average,1695
2,aws-aws-sdk-java-v2,2.9.9,5,euclidean,single,644
3,aws-aws-sdk-java-v2,2.9.9,5,l1,complete,1070
4,aws-aws-sdk-java-v2,2.9.9,5,l1,average,695


In [8]:
bunch_results = pd.DataFrame(results_data_arr_bunch)
bunch_results.columns = ['project_name', 'project_version', 'method', 'calculator', 'MoJo_results']
bunch_results.head()

,project_name,project_version,method,calculator,MoJo_results
0,cryptomator-cryptomator,1.5.6,hillclimbing,turbomqincrw,102
1,cryptomator-cryptomator,1.5.6,hillclimbing,basicmq,109
2,cryptomator-cryptomator,1.5.6,hillclimbing,turbomqincr,120
3,cryptomator-cryptomator,1.5.6,hillclimbing,turbomq,103
4,cryptomator-cryptomator,1.5.6,hillclimbing,turbomqw,116


In [9]:
bunch_results['project_name'].unique()

array(['cryptomator-cryptomator', 'dropwizard-dropwizard',
       'dropwizard-metrics', 'evant-gradle-retrolambda',
       'facebook-facebook-android-sdk',
       'facebook-facebook-java-business-sdk', 'facebook-fresco',
       'facebook-litho', 'facebook-react-native-fbsdk', 'google-cdep',
       'google-dagger', 'google-error-prone', 'google-exoplayer',
       'google-gitiles', 'google-openrtb-doubleclick', 'google-openrtb',
       'grpc-grpc-java', 'havarunner-havarunner', 'immutables-immutables',
       'ionic-team-capacitor', 'iSoron-uhabits', 'jankotek-mapdb',
       'java-native-accessjna', 'javafunk-funk', 'javaparser-javaparser',
       'jboss-switchyard-release', 'jenkinsci-jenkins', 'jhy-jsoup',
       'joelittlejohn-jsonschema2pojo', 'kijiproject-kiji-bento',
       'knowm-XChange', 'lettuce-io-lettuce-core', 'mockito-mockito',
       'mrniko-netty-socketio', 'mybatis-mybatis-3', 'naver-pinpoint',
       'netty-netty', 'oblac-jodd', 'openengsb-openengsb',
       'oracle-oci

In [10]:
proj_len_df = pd.DataFrame(len_data_arr)
proj_len_df.columns = ['project_name', 'len']
proj_len_df.head()

,project_name,len
0,aws-aws-sdk-java-v2,2168
1,bdew-neiaddons,38
2,bkromhout-realm-java,352
3,btraceio-btrace,495
4,bytedeco-javacpp,154


In [11]:
## Remove flyway error
agglo_results = agglo_results[agglo_results['project_name'] != 'flyway']
agglo_results = agglo_results[agglo_results['project_name'] != 'mrniko-netty-socketio']



In [12]:
#agglo_results.to_csv('agglo_results_21102020.csv', index=False)
#bunch_results.to_csv('bunch_results_21102020.csv', index=False)

In [13]:
proj_len_df.to_csv('proj_len.csv', index=False)

In [105]:
main_df_agglo = agglo_results.merge(proj_len_df, on='project_name', how='left')
main_df_agglo['len'] = main_df_agglo['len'].astype('int32')
main_df_agglo['cluster_division'] = main_df_agglo['cluster_division'].astype('int32')
main_df_agglo['MoJo_results'] = main_df_agglo['MoJo_results'].astype('int32')
main_df_agglo['num_clusters'] = main_df_agglo['len'] // main_df_agglo['cluster_division']
main_df_agglo['accuracy'] = 1-  (main_df_agglo['MoJo_results'] / main_df_agglo['len'])
main_df_agglo = main_df_agglo[main_df_agglo['affinity'] != 'l2']
main_df_agglo = main_df_agglo[main_df_agglo['affinity'] != 'l1']
main_df_agglo = main_df_agglo[main_df_agglo['accuracy'] != 1]
main_df_agglo['algo'] = 'algo_' + main_df_agglo['affinity'] + '_' + main_df_agglo['linkage'] + '_' + main_df_agglo['cluster_division'].astype(str)
main_df_agglo['Instances'] = main_df_agglo['project_name'] + '_' + main_df_agglo['project_version']
main_df_agglo = main_df_agglo.drop(columns=['project_name', 'project_version', 'cluster_division', 'affinity', 'linkage', 'MoJo_results', 'len', 'num_clusters'])

main_df_agglo = pd.pivot_table(main_df_agglo, values=['accuracy'], index=['Instances'], columns=['algo'])
main_df_agglo['Instances'] = main_df_agglo.index
main_df_agglo = main_df_agglo.reset_index(drop=True)
#main_df_agglo = main_df_agglo.groupby(['project_name', 'project_version']).apply(lambda x: x.nlargest(1, "accuracy")).reset_index(drop=True)
#print(main_df_agglo.columns.droplevel())
main_df_agglo.columns = main_df_agglo.columns.droplevel()

main_df_agglo.rename(columns={'':'Instances'}, inplace=True)
#main_df_agglo.columns = final_columns

main_df_agglo = main_df_agglo.fillna(0)

main_df_agglo.head(10)

algo,algo_cosine_average_10,algo_cosine_average_15,algo_cosine_average_20,algo_cosine_average_25,algo_cosine_average_5,algo_cosine_average_7,algo_cosine_complete_10,algo_cosine_complete_15,algo_cosine_complete_20,algo_cosine_complete_25,...,algo_manhattan_complete_25,algo_manhattan_complete_5,algo_manhattan_complete_7,algo_manhattan_single_10,algo_manhattan_single_15,algo_manhattan_single_20,algo_manhattan_single_25,algo_manhattan_single_5,algo_manhattan_single_7,Instances
0,0.198339,0.221863,0.244926,0.261531,0.210332,0.199262,0.418358,0.549815,0.613469,0.679889,...,0.808579,0.482934,0.623616,0.808579,0.844096,0.860701,0.870849,0.717712,0.769373,aws-aws-sdk-java-v2_2.9.24
1,0.191421,0.239852,0.242620,0.251384,0.204797,0.198801,0.414207,0.536900,0.608395,0.668358,...,0.808579,0.510609,0.622232,0.809502,0.845941,0.859317,0.869004,0.719096,0.772140,aws-aws-sdk-java-v2_2.9.25
2,0.191421,0.239852,0.242620,0.251384,0.204797,0.198801,0.414207,0.536900,0.608395,0.668358,...,0.808579,0.510609,0.622232,0.809502,0.845941,0.859317,0.869004,0.719096,0.772140,aws-aws-sdk-java-v2_2.9.26
3,0.193727,0.230166,0.248616,0.273524,0.213100,0.197878,0.398985,0.535517,0.601015,0.657288,...,0.799815,0.519834,0.636070,0.807657,0.843635,0.859317,0.869926,0.720480,0.770295,aws-aws-sdk-java-v2_2.9.3
4,0.193727,0.230166,0.248616,0.273524,0.213100,0.197878,0.398985,0.535517,0.601015,0.657288,...,0.799815,0.519834,0.636070,0.807657,0.843635,0.859317,0.869926,0.720480,0.770295,aws-aws-sdk-java-v2_2.9.4
5,0.193727,0.230166,0.248616,0.273524,0.213100,0.197878,0.398985,0.535517,0.601015,0.657288,...,0.799815,0.519834,0.636070,0.807657,0.843635,0.859317,0.869926,0.720480,0.770295,aws-aws-sdk-java-v2_2.9.5
6,0.193727,0.230166,0.248616,0.273524,0.213100,0.197878,0.398985,0.535517,0.601015,0.657288,...,0.799815,0.519834,0.636070,0.807657,0.843635,0.859317,0.869926,0.720480,0.770295,aws-aws-sdk-java-v2_2.9.6
7,0.193727,0.230166,0.248616,0.273524,0.213100,0.197878,0.398985,0.535517,0.601015,0.657288,...,0.799815,0.519834,0.636070,0.807657,0.843635,0.859317,0.869926,0.720480,0.770295,aws-aws-sdk-java-v2_2.9.7
8,0.193727,0.230166,0.248616,0.273524,0.213100,0.197878,0.398985,0.535517,0.601015,0.657288,...,0.799815,0.519834,0.636070,0.807657,0.843635,0.859317,0.869926,0.720480,0.770295,aws-aws-sdk-java-v2_2.9.8
9,0.192343,0.228321,0.249077,0.273063,0.211255,0.197878,0.402214,0.537823,0.605166,0.675738,...,0.798893,0.506458,0.635148,0.807196,0.843173,0.859317,0.870387,0.720941,0.768911,aws-aws-sdk-java-v2_2.9.9


In [106]:
main_df_bunch = bunch_results.merge(proj_len_df, on='project_name', how='left')
main_df_bunch['len'] = main_df_bunch['len'].astype('int32')
main_df_bunch['MoJo_results'] = main_df_bunch['MoJo_results'].astype('int32')
main_df_bunch['accuracy'] = 1-  (main_df_bunch['MoJo_results'] / main_df_bunch['len'])
#main_df_bunch = main_df_bunch.groupby(['project_name', 'project_version']).apply(lambda x: x.nlargest(1, "accuracy")).reset_index(drop=True)

main_df_bunch['algo'] = 'algo_' + main_df_bunch['method'] + '_' + main_df_bunch['calculator']
main_df_bunch['Instances'] = main_df_bunch['project_name'] + '_' + main_df_bunch['project_version']
main_df_bunch = main_df_bunch.drop(columns=['project_name', 'project_version', 'method', 'calculator', 'MoJo_results', 'len'])

main_df_bunch = pd.pivot_table(main_df_bunch, values=['accuracy'], index=['Instances'], columns=['algo'])
main_df_bunch['Instances'] = main_df_bunch.index
main_df_bunch = main_df_bunch.reset_index(drop=True)

main_df_bunch.columns = main_df_bunch.columns.droplevel()
main_df_bunch.rename(columns={'':'Instances'}, inplace=True)
main_df_bunch = main_df_bunch.fillna(0)




main_df_bunch.head(10)

algo,algo_exhaustive_basicmq,algo_exhaustive_turbomq,algo_exhaustive_turbomqincr,algo_exhaustive_turbomqincrw,algo_exhaustive_turbomqw,algo_ga_basicmq,algo_ga_turbomq,algo_ga_turbomqincr,algo_ga_turbomqincrw,algo_ga_turbomqw,algo_hillclimbing_basicmq,algo_hillclimbing_turbomq,algo_hillclimbing_turbomqincr,algo_hillclimbing_turbomqincrw,algo_hillclimbing_turbomqw,Instances
0,0.184685,0.202703,0.225225,0.216216,0.198198,0.225225,0.166667,0.198198,0.184685,0.211712,0.198198,0.180180,0.171171,0.193694,0.144144,cryptomator-cryptomator_1.5.0-beta3
1,0.184685,0.265766,0.216216,0.211712,0.229730,0.225225,0.225225,0.220721,0.184685,0.193694,0.225225,0.148649,0.171171,0.202703,0.189189,cryptomator-cryptomator_1.5.0-rc1
2,0.234234,0.211712,0.270270,0.157658,0.184685,0.270270,0.207207,0.207207,0.211712,0.189189,0.261261,0.252252,0.252252,0.211712,0.207207,cryptomator-cryptomator_1.5.0-rc2
3,0.207207,0.252252,0.193694,0.198198,0.234234,0.220721,0.198198,0.198198,0.202703,0.220721,0.171171,0.193694,0.193694,0.238739,0.238739,cryptomator-cryptomator_1.5.0-rc3
4,0.216216,0.225225,0.225225,0.202703,0.198198,0.198198,0.207207,0.225225,0.220721,0.256757,0.166667,0.157658,0.144144,0.220721,0.207207,cryptomator-cryptomator_1.5.1
5,0.333333,0.558559,0.518018,0.504505,0.481982,0.441441,0.554054,0.527027,0.463964,0.504505,0.468468,0.500000,0.441441,0.527027,0.536036,cryptomator-cryptomator_1.5.2
6,0.581081,0.477477,0.468468,0.536036,0.454955,0.445946,0.463964,0.522523,0.531532,0.527027,0.504505,0.486486,0.513514,0.436937,0.432432,cryptomator-cryptomator_1.5.3
7,0.351351,0.423423,0.472973,0.509009,0.545045,0.527027,0.481982,0.454955,0.540541,0.477477,0.463964,0.558559,0.531532,0.463964,0.572072,cryptomator-cryptomator_1.5.4
8,0.427928,0.513514,0.459459,0.513514,0.567568,0.436937,0.468468,0.486486,0.472973,0.554054,0.490991,0.563063,0.486486,0.522523,0.490991,cryptomator-cryptomator_1.5.5
9,0.477477,0.418919,0.468468,0.486486,0.545045,0.468468,0.536036,0.549550,0.468468,0.545045,0.509009,0.536036,0.459459,0.540541,0.477477,cryptomator-cryptomator_1.5.6


In [107]:
ck_max = pd.read_csv('ck_max_194.csv')
ck_std = pd.read_csv('ck_std_194.csv')
ck_mean = pd.read_csv('ck_mean_194.csv')
ck_sum = pd.read_csv('ck_sum_194.csv')

In [108]:
ck_max_column = ['project_name', 'project_version']
ck_std_column = ['project_name', 'project_version']
ck_mean_column = ['project_name', 'project_version']
ck_sum_column = ['project_name', 'project_version']

excluded_columns = ['project_name', 'project_version']



for element in ck_max.columns:
    #print(element)
    if element not in excluded_columns:
        ck_max_column.append('feature_' + element + '_max')
   
        
        
for element in ck_std.columns:
    #print(element)
    if element not in excluded_columns:
        ck_std_column.append('feature_' + element + '_std')
        
for element in ck_mean.columns:
    #print(element)
    if element not in excluded_columns:
        ck_mean_column.append('feature_' + element + '_mean')
        
for element in ck_sum.columns:
    #print(element)
    if element not in excluded_columns:
        ck_sum_column.append('feature_' + element + '_sum')
    
        
ck_max.columns = ck_max_column
ck_std.columns = ck_std_column
ck_sum.columns = ck_sum_column
ck_mean.columns = ck_mean_column




In [109]:
ck_max['Instances'] = ck_max['project_name'] + '_' + ck_max['project_version']
ck_std['Instances'] = ck_std['project_name'] + '_' + ck_std['project_version']
ck_sum['Instances'] = ck_sum['project_name'] + '_' + ck_sum['project_version']
ck_mean['Instances'] = ck_mean['project_name'] + '_' + ck_mean['project_version']

ck_max = ck_max.drop(columns=['project_name', 'project_version'])
ck_std = ck_std.drop(columns=['project_name', 'project_version'])
ck_sum = ck_sum.drop(columns=['project_name', 'project_version'])
ck_mean = ck_mean.drop(columns=['project_name', 'project_version'])


In [110]:
ck_max.head()

,feature_cbo_max,feature_wmc_max,feature_dit_max,feature_rfc_max,feature_lcom_max,feature_totalMethods_max,feature_staticMethods_max,feature_publicMethods_max,feature_privateMethods_max,feature_protectedMethods_max,...,feature_assignmentsQty_max,feature_mathOperationsQty_max,feature_variablesQty_max,feature_maxNestedBlocks_max,feature_anonymousClassesQty_max,feature_subClassesQty_max,feature_lambdasQty_max,feature_uniqueWordsQty_max,feature_modifiers_max,Instances
0,79.0,203.0,17.0,200.0,6811.0,131.0,58.0,123.0,50.0,23.0,...,353.0,195.0,297.0,6.0,21.0,13.0,0.0,573.0,1025.0,JodaOrg-joda-time_v2.10.6
1,78.0,203.0,17.0,194.0,6578.0,129.0,71.0,121.0,50.0,23.0,...,353.0,195.0,292.0,6.0,21.0,13.0,0.0,553.0,1025.0,JodaOrg-joda-time_v2.2
2,78.0,203.0,17.0,194.0,6811.0,129.0,58.0,121.0,50.0,23.0,...,353.0,195.0,292.0,6.0,21.0,13.0,0.0,554.0,1025.0,JodaOrg-joda-time_v2.3
3,78.0,203.0,17.0,195.0,6811.0,131.0,58.0,123.0,50.0,23.0,...,353.0,195.0,297.0,6.0,21.0,13.0,0.0,554.0,1025.0,JodaOrg-joda-time_v2.4
4,78.0,203.0,17.0,195.0,6811.0,131.0,58.0,123.0,50.0,23.0,...,353.0,195.0,297.0,6.0,21.0,13.0,0.0,561.0,1025.0,JodaOrg-joda-time_v2.5


In [111]:
main_df_agglo = main_df_agglo.merge(ck_max, how='left', on=['Instances'])
main_df_agglo = main_df_agglo.merge(ck_mean, how='left', on=['Instances'])
main_df_agglo = main_df_agglo.merge(ck_std, how='left', on=['Instances'])
main_df_agglo = main_df_agglo.merge(ck_sum, how='left', on=['Instances'])

main_df_agglo.head()

,algo_cosine_average_10,algo_cosine_average_15,algo_cosine_average_20,algo_cosine_average_25,algo_cosine_average_5,algo_cosine_average_7,algo_cosine_complete_10,algo_cosine_complete_15,algo_cosine_complete_20,algo_cosine_complete_25,...,feature_numbersQty_sum,feature_assignmentsQty_sum,feature_mathOperationsQty_sum,feature_variablesQty_sum,feature_maxNestedBlocks_sum,feature_anonymousClassesQty_sum,feature_subClassesQty_sum,feature_lambdasQty_sum,feature_uniqueWordsQty_sum,feature_modifiers_sum
0,0.198339,0.221863,0.244926,0.261531,0.210332,0.199262,0.418358,0.549815,0.613469,0.679889,...,5625.0,16197.0,2432.0,14775.0,1243.0,330.0,835.0,1878.0,124807.0,128965.0
1,0.191421,0.239852,0.242620,0.251384,0.204797,0.198801,0.414207,0.536900,0.608395,0.668358,...,5631.0,16224.0,2433.0,14796.0,1245.0,333.0,837.0,1889.0,125059.0,128952.0
2,0.191421,0.239852,0.242620,0.251384,0.204797,0.198801,0.414207,0.536900,0.608395,0.668358,...,5631.0,16224.0,2433.0,14796.0,1245.0,333.0,837.0,1889.0,125059.0,128952.0
3,0.193727,0.230166,0.248616,0.273524,0.213100,0.197878,0.398985,0.535517,0.601015,0.657288,...,5559.0,16029.0,2396.0,14620.0,1236.0,330.0,826.0,1849.0,123843.0,127845.0
4,0.193727,0.230166,0.248616,0.273524,0.213100,0.197878,0.398985,0.535517,0.601015,0.657288,...,5559.0,16029.0,2396.0,14620.0,1236.0,330.0,826.0,1849.0,123843.0,127845.0


In [112]:
main_df_bunch = main_df_bunch.merge(ck_max, how='left', on=['Instances'])
main_df_bunch = main_df_bunch.merge(ck_mean, how='left', on=['Instances'])
main_df_bunch = main_df_bunch.merge(ck_std, how='left', on=['Instances'])
main_df_bunch = main_df_bunch.merge(ck_sum, how='left', on=['Instances'])

main_df_bunch.head()

,algo_exhaustive_basicmq,algo_exhaustive_turbomq,algo_exhaustive_turbomqincr,algo_exhaustive_turbomqincrw,algo_exhaustive_turbomqw,algo_ga_basicmq,algo_ga_turbomq,algo_ga_turbomqincr,algo_ga_turbomqincrw,algo_ga_turbomqw,...,feature_numbersQty_sum,feature_assignmentsQty_sum,feature_mathOperationsQty_sum,feature_variablesQty_sum,feature_maxNestedBlocks_sum,feature_anonymousClassesQty_sum,feature_subClassesQty_sum,feature_lambdasQty_sum,feature_uniqueWordsQty_sum,feature_modifiers_sum
0,0.184685,0.202703,0.225225,0.216216,0.198198,0.225225,0.166667,0.198198,0.184685,0.211712,...,268.0,1132.0,96.0,1158.0,100.0,7.0,44.0,111.0,6898.0,17591.0
1,0.184685,0.265766,0.216216,0.211712,0.229730,0.225225,0.225225,0.220721,0.184685,0.193694,...,268.0,1157.0,95.0,1184.0,102.0,7.0,46.0,112.0,7063.0,18618.0
2,0.234234,0.211712,0.270270,0.157658,0.184685,0.270270,0.207207,0.207207,0.211712,0.189189,...,268.0,1158.0,95.0,1185.0,102.0,7.0,46.0,113.0,7073.0,18618.0
3,0.207207,0.252252,0.193694,0.198198,0.234234,0.220721,0.198198,0.198198,0.202703,0.220721,...,268.0,1158.0,95.0,1185.0,102.0,7.0,46.0,113.0,7075.0,18618.0
4,0.216216,0.225225,0.225225,0.202703,0.198198,0.198198,0.207207,0.225225,0.220721,0.256757,...,268.0,1164.0,97.0,1191.0,101.0,7.0,46.0,114.0,7136.0,18619.0


In [114]:
main_df_agglo.to_csv('agglo_results_matilda_19112020.csv', index=False)
main_df_bunch.to_csv('bunch_results_matilda_19112020.csv', index=False)